In [6]:
test_input = """Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3"""

In [7]:
import re
import numpy as np
import math
import itertools

p = "x=(-?\d+), y=(-?\d+)"

In [8]:
def parse_input(s):
    return [tuple((int(a), int(b)) for (a, b) in re.findall(p, l)) for l in s.split("\n")]

In [9]:
def man_d(t1, t2):
    return abs(t1[0]-t2[0]) + abs(t1[1]-t2[1])

In [10]:
def p1(data, Y):
    min_unoccupied = math.inf
    max_unoccupied = -math.inf

    # intervals = []

    for sensor, beacon in data:
        dist = man_d(sensor, beacon)

        y_dist = abs(sensor[1] - Y)
        if y_dist > dist:
            # too far to be relevant to Y line
            pass
        else:
            # could be relevant.
            # relevance range:
            sensor_x = sensor[0]
            remaining_dist = dist - y_dist

            min_reach = sensor_x - remaining_dist
            max_reach = sensor_x + remaining_dist

            # intervals.append((min_reach, max_reach))

            if min_reach < min_unoccupied:
                min_unoccupied = min_reach

            if max_reach > max_unoccupied:
                max_unoccupied = max_reach
    
    return max_unoccupied - min_unoccupied

In [11]:
# Y = 10
Y = 2_000_000

In [12]:
p1(parse_input(open("inputs/15").read()), Y=2_000_000)

4748135

In [13]:
# UGLY!!

def merge(t1, t2):
    (a, b) = t1
    (c, d) = t2

    if a >= c and a <= d:
        overlap = True
    elif b >= c and b <= d:
        overlap = True
    elif c >= a and c <= b:
        overlap = True
    elif d >= a and d <= b:
        overlap = True
    else:
        overlap = False
    
    if overlap:
        return (min(a, c), max(b, d))

def merge_all(tups):
    n_tups = len(tups)

    while True:
        for (i, j) in itertools.combinations(range(n_tups), 2):
            m = merge(tups[i], tups[j])

            if m:
                tups[i] = m
                del tups[j]
                n_tups -= 1
                break
        else:
            # if we got through without merging we're done
            break
    
    return tups

In [14]:
merge((0, 5), (6, 10))

In [36]:
def p2(data, y_limit):
    for y in range(y_limit+1):
        intervals = []

        for sensor, beacon in data:
            dist = man_d(sensor, beacon)

            y_dist = abs(sensor[1] - y)

            if y_dist <= dist:
                # could be relevant.
                # relevance range:
                sensor_x = sensor[0]
                remaining_dist = dist - y_dist

                min_reach = sensor_x - remaining_dist
                max_reach = sensor_x + remaining_dist

                intervals.append((min_reach, max_reach))

        merged_intervals = merge_all(intervals) 

        if len(merged_intervals) > 1:
            assert len(merged_intervals) == 2

            (a, b), (c, d) = sorted(merged_intervals)

            # gap in middle
            assert c - b == 2

            return 4000000 * (c - 1) + y

In [37]:
p2(parse_input(test_input), y_limit=20)

56000011

In [35]:
p2(parse_input(open("inputs/15").read()), y_limit=4000000)

13743542639657